In [44]:
import shap
import transformers
import numpy as np
from Model.evaluation_model import word_similarity_score
import spacy

from Data.data_loader import load_data

In [45]:
language_model = "en_core_web_lg"
nlp = spacy.load(language_model)

open_word_list = ['open', 'outgoing', 'curious', 'open-minded', 'broad-minded', 'honest', 'empathetic', 'respectful',
                  'positivity', 'emotional intelleligence', 'interest', 'interested', 'adapting', 'informative',
                  'sharing', 'feedback', 'honesty', 'trust', 'valuing', 'diversity', 'perspective']

open_words_string = " ".join(open_word_list)

In [46]:
statement = "He is the most open person I have ever seen. However, he is not creative."

def get_adjectives(statement):
    sent_list = list(nlp(statement).sents)
    adj_list = []
    for sent in sent_list:
        adj_list = adj_list + [t.text for t in sent.as_doc() if t.pos_ in ['ADJ']]
    return adj_list


adjectives = get_adjectives(statement)

In [53]:
model = transformers.pipeline('sentiment-analysis', return_all_scores=True)
explainer = shap.Explainer(model)

statement = "He is the most open person I have ever seen. However, he is not creative."
def split_word_string(word_string=statement, pad=True):

    splited_word_list = word_string.split(" ")
    if pad:
        splited_word_list = [""] + [word +" " for word in splited_word_list] + [""]

    return splited_word_list

def modify_shap_value(explainer=explainer, statement=statement, sim_word=open_words_string):
    shap_values = explainer([statement])
    shap_value = shap_values[0, :, 'POSITIVE']

    modified_shap_value = shap_value
    splited_statement = split_word_string(statement)

    base_value = word_similarity_score(splited_statement, word=sim_word)
    modified_shap_value.base_values = base_value
    modified_shap_value.values = np.array([word_similarity_score(single_word, word=sim_word) - base_value for single_word in splited_statement])
    modified_shap_value.values[0] = modified_shap_value.values[-1] = 0
    modified_shap_value.hierarchical_values = modified_shap_value.values

    modified_shap_value.data = np.array(splited_statement)

    return modified_shap_value


In [54]:
new_shapp_value = modify_shap_value(explainer,  statement,sim_word="open")
print(new_shapp_value)


.values =
array([ 0.        , -0.04967574, -0.03871701,  0.03706433, -0.06039805,
        0.56527963, -0.09766921, -0.12036045,  0.00263358, -0.09322166,
        0.0012357 , -0.07331458, -0.04967574, -0.03871701,  0.03369131,
       -0.01815511,  0.        ])

.base_values =
0.4347203705750323

.data =
array(['', 'He ', 'is ', 'the ', 'most ', 'open ', 'person ', 'I ',
       'have ', 'ever ', 'seen. ', 'However, ', 'he ', 'is ', 'not ',
       'creative. ', ''], dtype='<U10')


In [55]:
new_shapp_value.hierarchical_values = new_shapp_value.values
print(new_shapp_value.hierarchical_values)
print(new_shapp_value.values)

[ 0.         -0.04967574 -0.03871701  0.03706433 -0.06039805  0.56527963
 -0.09766921 -0.12036045  0.00263358 -0.09322166  0.0012357  -0.07331458
 -0.04967574 -0.03871701  0.03369131 -0.01815511  0.        ]
[ 0.         -0.04967574 -0.03871701  0.03706433 -0.06039805  0.56527963
 -0.09766921 -0.12036045  0.00263358 -0.09322166  0.0012357  -0.07331458
 -0.04967574 -0.03871701  0.03369131 -0.01815511  0.        ]


In [56]:
shap.plots.text(new_shapp_value, xmin=0)
